In [1]:
import torch
import torch.nn as nn

import numpy as np

from functorch import make_functional, vmap, grad, jacrev
import functools

from pyDOE import lhs

device = torch.device('cpu')

torch.set_default_dtype(torch.float64)

In [2]:
class Plain(nn.Module):
    
    def __init__(self, in_dim , h_dim , out_dim):
        super().__init__()
        self.ln1 = nn.Linear( in_dim , h_dim )
        self.act1 =nn.Sigmoid()
        self.ln2 = nn.Linear( h_dim , out_dim , bias=False )
        
    def forward(self, x):
        out = self.ln1(x)
        out = self.act1(out)
        out = self.ln2(out)
        return out

In [3]:
def compute_loss_Res(func_params, X_inner, Rf_inner):

    def f(x, func_params):
        output = func_model(func_params, x)
        return output.squeeze(0)
    
    grad2_f = (jacrev(grad(f)))(X_inner, func_params)
    dudX2 = (torch.diagonal(grad2_f))
    
    laplace = (dudX2[0] + dudX2[1])
    
    loss_Res = laplace - Rf_inner

    return loss_Res.flatten()


def compute_loss_b(func_params, X_bd, U_bd):

    def f(x, func_params):
        output = func_model(func_params, x)
        return output.squeeze(0)
    
    u_pred = f(X_bd, func_params)
    loss_b = u_pred - U_bd
        
    return loss_b.flatten()

def compute_loss_j(func_params, X_ij, Uj_ij):

    def f(x, func_params):
        output = func_model(func_params, x)
        return output.squeeze(0)
    
    X_ij=X_ij.reshape(len(X_ij), 1)

    ij_outer = torch.cat((X_ij[0], X_ij[1], 1.0+0.0*X_ij[0]), 0)
    ij_inner = torch.cat((X_ij[0], X_ij[1], -1.0+0.0*X_ij[0]), 0)

    u_ij_outer = f(ij_outer, func_params)
    u_ij_inner = f(ij_inner, func_params)
    
    ij_pred = u_ij_outer - u_ij_inner
    
    loss_j = ij_pred - Uj_ij
        
    return loss_j.flatten()

def compute_loss_normal_jump(func_params, X_ij, Normal_ij, Unj_ij):

    def f(x, func_params):
        output = func_model(func_params, x)
        return output.squeeze(0)
    
    X_ij=X_ij.reshape(len(X_ij), 1)
    
    ij_outer = torch.cat((X_ij[0], X_ij[1], 1.0+0.0*X_ij[0]), 0)
    ij_inner = torch.cat((X_ij[0], X_ij[1], -1.0+0.0*X_ij[0]), 0)

    normal_x = Normal_ij[0]
    normal_y = Normal_ij[1]
    
    grad_f_outer = (grad(f))(ij_outer, func_params)
    df_outer = (grad_f_outer)
    Normal_outer = normal_x*df_outer[0] + normal_y*df_outer[1]
    grad_f_inner = (grad(f))(ij_inner, func_params)
    df_inner = (grad_f_inner)
    Normal_inner = normal_x*df_inner[0] + normal_y*df_inner[1]
    
    normal_jump_pred = 1.0e-3*Normal_outer - Normal_inner

    loss_normal_jump = normal_jump_pred - Unj_ij
    
    return loss_normal_jump.flatten()

In [4]:
# exact_u = exact solution
def exact_u(x, y, z):
    u1 = np.sin(x)*np.sin(y)
    un1 = np.exp(x+y)
    eu = u1*(1.0+z)/2.0 + un1*(1.0-z)/2.0
    return eu

# rhs_f = right hand side function
def rhs_f(x, y, z):
    f1 = -2.0*np.sin(x)*np.sin(y)
    fn1 = 2.0*np.exp(x+y)
    rf = f1*(1.0+z)/2.0 + fn1*(1.0-z)/2.0
    return rf

# normal_u = \nabla u \dot n, normal derivative of u, only defined on the interface
def normal_u(x, y, z, normal_x, normal_y):
    u1x = np.cos(x)*np.sin(y)
    u1y = np.sin(x)*np.cos(y)
    u1 = normal_x*u1x + normal_y*u1y
    un1x = np.exp(x+y)
    un1y = np.exp(x+y)
    un1 = normal_x*un1x + normal_y*un1y
    nu = u1*(1.0+z)/2.0 + un1*(1.0-z)/2.0
    return nu

def sign_x(x, y):
    z = 0.0*x + 1.0
    for i in range(len(z)):
        dist = np.sqrt(x[i]**2+y[i]**2)
        theta = np.arctan(y[i]/x[i])
        ri = 0.4-0.2*np.cos(5.0*theta)
        if dist < ri:
            z[i] = -1.0
    return z

In [5]:
# number of grid points
N_inner = 8

# Training points

## X_inner: points inside the domain
X_inner = 2.0*lhs(2, 2*N_inner**2) - 1.0
x = X_inner[:,0:1]
y = X_inner[:,1:2]
dist = np.sqrt(x**2+y**2)
theta_inner = np.arctan(y/x)
rr = 1.0-0.3*np.cos(5.0*theta_inner)
dd = np.where(dist>rr)
x = np.delete(x, dd[0], 0)
y = np.delete(y, dd[0], 0)
z = sign_x(x, y)
Rf_inner = rhs_f(x, y, z)
X_inner = np.hstack((x, y, z))

## X_bd: points at the boundary
theta_bd = 2.0*np.pi*lhs(1, 4*N_inner)
r_bd = 1.0-0.3*np.cos(5.0*theta_bd)
x_bd = r_bd*np.cos(theta_bd)
y_bd = r_bd*np.sin(theta_bd)
X_bd = np.hstack([x_bd, y_bd])
X_bd = np.hstack((X_bd, 1.0+0.0*X_bd[:,0:1]))

## U_bd: function values on the boundary
x = X_bd[:,0:1]
y = X_bd[:,1:2]
z = 0.0*x + 1.0
U_bd = exact_u(x, y, z)

## X_ij: points on the interior interface
theta = 2.0*np.pi*lhs(1, 4*N_inner)
ri = 0.4-0.2*np.cos(5.0*theta)
dri = np.sin(5.0*theta)
x_ij = ri*np.cos(theta)
y_ij = ri*np.sin(theta)
X_ij = np.hstack([x_ij, y_ij])

## normal vector
normal_x = dri*np.sin(theta) + ri*np.cos(theta)
normal_y = -dri*np.cos(theta) + ri*np.sin(theta)
dist = np.sqrt(normal_x**2+normal_y**2)
normal_x = normal_x/dist
normal_y = normal_y/dist
Normal_ij = np.hstack((normal_x, normal_y))

## Uj_ij: function jump on the interior interface
Uj_ij = exact_u(x_ij, y_ij, 0.0*x_ij+1.0) - exact_u(x_ij, y_ij, 0.0*x_ij-1.0)

# beta_plus
beta_plus = 1.0e-3
## Unj_ij: normal jump on the interior interface, totally 4*N_inner points
Unj_ij = beta_plus*normal_u(x_ij, y_ij, 0.0*x_ij+1.0, normal_x, normal_y) - normal_u(x_ij, y_ij, 0.0*x_ij-1.0, normal_x, normal_y)

In [6]:
# single-layer model
model = Plain(3, 20, 1).to(device)
print(model)

# Make model a functional
func_model, func_params = make_functional(model)

Plain(
  (ln1): Linear(in_features=3, out_features=20, bias=True)
  (act1): Sigmoid()
  (ln2): Linear(in_features=20, out_features=1, bias=False)
)


In [7]:
X_bd = torch.from_numpy(X_bd).requires_grad_(True).double().to(device)
U_bd = torch.from_numpy(U_bd).double().to(device)
X_inner = torch.from_numpy(X_inner).requires_grad_(True).double().to(device)
Rf_inner = torch.from_numpy(Rf_inner).double().to(device)
X_ij = torch.from_numpy(X_ij).requires_grad_(True).double().to(device)
Normal_ij = torch.from_numpy(Normal_ij).double().to(device)
Uj_ij = torch.from_numpy(Uj_ij).double().to(device)
Unj_ij = torch.from_numpy(Unj_ij).double().to(device)

In [8]:
LM_iter = 2000
mu_update = 2 # update \mu every mu_update iterations
div_factor = 1.3 # \mu <- \mu/div_factor when loss decreases
mul_factor = 3 # \mu <- mul_factor*\mu when loss incerases

mu = 10**5
loss_sum_old = 10**5
itera = 0

In [9]:
%%time
for step in range(LM_iter+1):
    # Put into loss functional to get L_vec
    L_vec_res = vmap(compute_loss_Res, (None, 0, 0))(func_params, X_inner, Rf_inner)
    L_vec_b = vmap(compute_loss_b, (None, 0, 0))(func_params, X_bd, U_bd)
    L_vec_j = vmap(compute_loss_j, (None, 0, 0))(func_params, X_ij, Uj_ij)
    L_vec_nj = vmap(compute_loss_normal_jump, (None, 0, 0, 0))(func_params, X_ij, Normal_ij, Unj_ij)

    L_vec_res = L_vec_res/np.sqrt(N_inner**2)
    L_vec_b = L_vec_b/np.sqrt(4.0*N_inner)
    L_vec_j = L_vec_j/np.sqrt(4.0*N_inner)
    L_vec_nj = L_vec_nj/np.sqrt(4.0*N_inner)
    loss = torch.sum(L_vec_res**2) + torch.sum(L_vec_b**2) + torch.sum(L_vec_j**2) + torch.sum(L_vec_nj**2)

    # Consturct J for domain points
    # (None, 0 ,0): func_params: no batch. data_d: batch wrt shape[0] (data[i, :]). force_value: batch wrt shape[0] (force_value[i,:])
    
    per_sample_grads = vmap(jacrev(compute_loss_Res), (None, 0, 0))(func_params, X_inner, Rf_inner)
    cnt = 0
    for g in per_sample_grads: 
        g = g.detach()
        J_d_res = g.view(len(g), -1) if cnt == 0 else torch.hstack([J_d_res, g.view(len(g), -1)])
        cnt = 1
    
    per_sample_grads = vmap(jacrev(compute_loss_b), (None, 0, 0))(func_params, X_bd, U_bd)
    cnt = 0
    for g in per_sample_grads: 
        g = g.detach()
        J_d_b = g.view(len(g), -1) if cnt == 0 else torch.hstack([J_d_b, g.view(len(g), -1)])
        cnt = 1
        
    per_sample_grads = vmap(jacrev(compute_loss_j), (None, 0, 0))(func_params, X_ij, Uj_ij)
    cnt = 0
    for g in per_sample_grads: 
        g = g.detach()
        J_d_j = g.view(len(g), -1) if cnt == 0 else torch.hstack([J_d_j, g.view(len(g), -1)])
        cnt = 1
        
    per_sample_grads = vmap(jacrev(compute_loss_normal_jump), (None, 0, 0, 0))(func_params, X_ij, Normal_ij, Unj_ij)
    cnt = 0
    for g in per_sample_grads: 
        g = g.detach()
        J_d_nj = g.contiguous().view(len(g), -1) if cnt == 0 else torch.hstack([J_d_nj, g.view(len(g), -1)])
        cnt = 1

    # cat J_d and J_b into J
    J_mat = torch.cat((J_d_res, J_d_b, J_d_j, J_d_nj))
    L_vec = torch.cat((L_vec_res, L_vec_b, L_vec_j, L_vec_nj))

    # update lambda
    I = torch.eye((J_mat.shape[1])).to(device)

    with torch.no_grad():
        J_product = J_mat.t()@J_mat
        rhs = -J_mat.t()@L_vec
        with torch.no_grad():
            dp = torch.linalg.solve(J_product + mu*I, rhs)

        # update parameters
        cnt=0
        for p in func_params:
            mm=torch.Tensor([p.shape]).tolist()[0]
            num=int(functools.reduce(lambda x,y:x*y,mm,1))
            p+=dp[cnt:cnt+num].reshape(p.shape)
            cnt+=num

        itera += 1
        if step % mu_update == 0:
            #if loss_sum_check < loss_sum_old:
            if loss < loss_sum_old:
                mu = max(mu/div_factor, 10**(-9))
            else:
                mu = min(mul_factor*mu, 10**(8))
            loss_sum_old = loss
                
        if step%100 == 0:
            print(
                    'Iter %d, Loss_Res: %.5e, mu: %.5e' % (itera, loss.item(), mu)
                )            

        if step == LM_iter or loss.item()<10**(-12):
            break

Iter 1, Loss_Res: 4.12813e+00, mu: 7.69231e+04
Iter 101, Loss_Res: 1.43960e-01, mu: 1.54486e-01
Iter 201, Loss_Res: 1.00092e-05, mu: 7.17761e-05
Iter 301, Loss_Res: 6.76532e-09, mu: 1.30057e-07
Iter 401, Loss_Res: 1.05769e-09, mu: 2.12625e-07
Iter 501, Loss_Res: 8.29961e-10, mu: 3.47610e-07
Iter 601, Loss_Res: 7.09930e-10, mu: 5.68291e-07
Iter 701, Loss_Res: 6.36720e-10, mu: 2.38223e-07
Iter 801, Loss_Res: 5.84455e-10, mu: 3.89460e-07
Iter 901, Loss_Res: 5.35112e-10, mu: 6.36710e-07
Iter 1001, Loss_Res: 4.83519e-10, mu: 2.66904e-07
Iter 1101, Loss_Res: 4.25624e-10, mu: 4.36349e-07
Iter 1201, Loss_Res: 3.45601e-10, mu: 1.82914e-07
Iter 1301, Loss_Res: 2.62951e-10, mu: 2.99038e-07
Iter 1401, Loss_Res: 1.91480e-10, mu: 4.88882e-07
Iter 1501, Loss_Res: 1.35271e-10, mu: 2.04936e-07
Iter 1601, Loss_Res: 9.26935e-11, mu: 3.35040e-07
Iter 1701, Loss_Res: 5.64079e-11, mu: 1.40446e-07
Iter 1801, Loss_Res: 2.73825e-11, mu: 5.88741e-08
Iter 1901, Loss_Res: 1.30748e-11, mu: 9.62505e-08
Iter 2001, L

In [10]:
# number of test points
N_test = 12800

# Error on the interior points
X_inn = 2.0*lhs(2, 2*N_test) - 1.0
xx = X_inn[:,0:1]
yy = X_inn[:,1:2]
dist = np.sqrt(xx**2+yy**2)
theta_inn = np.arctan(yy/xx)
rr = 1.0-0.3*np.cos(5.0*theta_inn)
dd = np.where(dist>rr)
xx = np.delete(xx, dd[0], 0)
yy = np.delete(yy, dd[0], 0)
zz = sign_x(xx, yy)
Exact_test = exact_u(xx, yy, zz)
X_inn = np.hstack((xx, yy, zz))

X_inn_torch = torch.tensor(X_inn).double().to(device)
u_pred = func_model(func_params, X_inn_torch).detach().cpu().numpy()

error = np.absolute(u_pred - Exact_test)

error_u_inf = np.linalg.norm(error, np.inf)
print('Error u (absolute inf-norm): %e' % (error_u_inf))
error_u_2 = np.linalg.norm(error,2)/np.sqrt(N_test)
print('Error u (absolute 2-norm): %e' % (error_u_2))

Error u (absolute inf-norm): 3.584442e-06
Error u (absolute 2-norm): 4.650794e-07
